In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output/cities.csv"

# define url
query_url = f"http://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}&q="

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

## Perform API Calls

### Perform a weather check on each city using a series of successive API calls.
### Include a print log of each city as it'sbeing processed (with the city number and city name).

In [ ]:
# set lists for the dataframe
city_two = []
cloudinesses = []
dates = []
humidities = []
lats = []
lngs = []
max_temps = []
wind_speeds = []
countries = []

# set initial count quantities for organization
count_one = 0
set_one = 1

# loops for creating dataframe columns
for city in cities:
    try:
        response = requests.get(query_url + city.replace(" ","&")).json()
        cloudinesses.append(response['clouds']['all'])
        countries.append(response['sys']['country'])
        dates.append(response['dt'])
        humidities.append(response['main']['humidity'])
        lats.append(response['coord']['lat'])
        lngs.append(response['coord']['lon'])
        max_temps.append(response['main']['temp_max'])
        wind_speeds.append(response['wind']['speed'])
        if count_one > 48:
            count_one = 1
            set_one += 1
            city_two.append(city)
        else:
            count_one += 1
            city_two.append(city)
        print(f"Processing Record {count_one} of Set {set_one} | {city}")
    except Exception:
        print("City not found. Skipping...")
print("-----------------------------") 
print("Data Retrieval Complete")  
print("---------------------------") 

## Convert Raw Data to DataFrame

##### Export the city data into a .csv.Display the DataFrame

In [ ]:
# create a dataframe
weather_df = {
    "City":city_two,
    "Cloudiness":cloudinesses,
    "Country":countries,
    "Date":dates,
    "Humidity":humidities,
    "Lat":lats,
    "Lng":lngs,
    "Max Temp":max_temps,
    "Wind Speed":wind_speeds
}

In [ ]:
weather = pd.DataFrame(weather_df)
weather.head()

In [ ]:
weather.describe()

In [ ]:
#exporting to csv
weather.to_csv("output/City_Weather_Data.csv")

##  Inspect the data and remove the cities where the humidity > 100%.

In [ ]:
Humidity_greater = weather.loc[weather["Humidity"] > 100]
Humidity_greater

In [ ]:
## Skiping this step as there are no cities that have humidity > 100%.

## Plotting the Data

##### Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
##### Save the plotted figures as .pngs.

### Latitude vs. Temperature Plot

In [ ]:
plt.scatter(weather["Lat"],weather["Max Temp"],edgecolors="black",facecolors="skyblue")
plt.title("City Latitude vs. Max Temperature (08/15/22)")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")
plt.grid (b=True,which="major",axis="both",linestyle="-",color="grey")
plt.savefig("output/fig1.png")
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
plt.scatter(weather["Lat"],weather["Humidity"],edgecolors="black",facecolors="skyblue")
plt.title("City Latitude vs. Humidity (08/15/22)")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.ylim(15,300)
plt.grid (b=True,which="major",axis="both",linestyle="-",color="lightgrey")
plt.savefig("output/fig2.png")
plt.show()

##  Latitude vs. Cloudiness Plot

In [ ]:
plt.scatter(weather["Lat"],weather["Cloudiness"],edgecolors="black",facecolors="skyblue")
plt.title("City Latitude vs. Cloudiness (08/15/22)")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.grid (b=True,which="major",axis="both",linestyle="-",color="lightgrey")
plt.savefig("output/fig3.png")
plt.show()


## Latitude vs. Wind Speed Plot

In [ ]:
plt.scatter(weather["Lat"],weather["Wind Speed"],edgecolors="black",facecolors="skyblue")
plt.title("City Latitude vs. Wind Speed (08/15/22)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.ylim(-2,30)
plt.grid (b=True,which="major",axis="both",linestyle="-",color="lightgrey")
plt.savefig("output/fig4.png")
plt.show()